## loading data/libs

In [ ]:
import pandas as pd
import calendar
from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.plotting import figure
from bokeh.io import hplot
output_notebook() 

In [ ]:
INPUT="data/device_failure.csv" 
dataset = pd.read_csv(INPUT,index_col=[0,1],parse_dates=[0])

In [ ]:
label_dset = dataset[["failure"]]


### checking devices

In [ ]:
total_failures_per_device = label_dset.groupby(level=1).agg(sum)
total_failures_per_device["failure"].value_counts()

Each device fail at least once

~10% device failing

'only' 106 positive points

### checking Dates

In [ ]:
dates = label_dset.index.get_level_values(0)
print "Range: from %s to %s" % (dates.min(), dates.max())


total_failures_per_date = label_dset.groupby(level=0).agg(sum)
print
print " n failures per date"
print str(total_failures_per_date["failure"].value_counts())
print
print "total: %i failures for %i days" % (total_failures_per_date["failure"].sum(), 
                                          total_failures_per_date[total_failures_per_date.failure >0].count())

In [ ]:
from bokeh.plotting import figure
data =total_failures_per_date.resample("M").sum() 
test = label_dset.reset_index("device").resample("M").agg(lambda d : d.nunique())
data["n_devices"] = test["device"]
data["failure_ratio_percent"] = data["failure"] / data["n_devices"] * 100
data.index = (calendar.month_abbr[i] for i in data.index.month)
l = Line(
    data["failure_ratio_percent"],
    title="failures per Month",
    ylabel="% failure", 
    xlabel="month"
)
show(l)

In [ ]:
l = Line(
    data["n_devices"],
    title="n devices seen per Month",
    ylabel="n_devices", 
    xlabel="month"
)
show(l)

In [ ]:
weekday_dset = total_failures_per_date.copy()
weekday_dset.index = ["%i:%s" % (i,calendar.day_name[i]) for i in total_failures_per_date.index.weekday]

per_day = weekday_dset.groupby(level=0).sum()

print "failures per weekday"

per_day.sort_index()


In [ ]:
from tabulate import tabulate
# uncomment to print "markdown-compatible" output
#d = per_day.sort_index()
#print tabulate(d , headers = ["weekday","NB failures" ],tablefmt="pipe")

- Long term trend with more failures in the past
- Less  failures over the weekend
- The absence of weekend could be explained by maintenance hapening only during workweek (hence explaing more failures on monday


### Per Device description 

In [ ]:
import numpy as np
dates = label_dset.swaplevel().reset_index("date")
dd= dates["date"]
devices = pd.DataFrame({"min_date":dd.groupby(level=0).min(),"failure":dates["failure"].groupby(level=0).sum()})
devices["max_date"] =  dd.groupby(level=0).max()
devices["n_lines"] = dd.groupby(level=0).count()
devices["n_days"] = (devices["max_date"] - devices["min_date"] ) /np.timedelta64(1, 'D') +1
devices["missing_values"] = devices["n_days"] - devices["n_lines"]

In [ ]:
devices["min_date"].value_counts()

### checking the nb devices per month. this is better done above

In [ ]:
#pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices = pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices.index = [calendar.month_abbr[i] for i in montlhy_devices.index]
montlhy_devices

### bucketing the n devices with missing day data

In [ ]:
i = ( (devices["missing_values"] //20)*20).value_counts()
#i = ( (devices["missing_values"])).value_counts(bins=10)
i.index.name = "n missing days"
pd.DataFrame({"n devices":i.sort_index()})

In [ ]:
i = devices["n_days"].value_counts(bins=10).sort_index()
i.index.name='n_days'
b = Bar(pd.DataFrame(
    {"n_devices":i}),
    xlabel="n days",
   title="devices distributed by ndays"
       )
show(b)

In [ ]:
failing_devices = devices[devices["failure"]>0].index
failing_devices_t = pd.DataFrame({"failure":label_dset["failure"].unstack().filter(items=failing_devices).unstack()}).dropna()
def max_date(date):
    return np.max(date)

def failing_date(date):
    data = withdate.ix[date.index]
    return data[data["failure"]>0]["date"][0]

withdate = failing_devices_t.reset_index(level=1)
max_vs_failingdates = withdate.groupby(level=0).agg( {"date": [ max_date, failing_date ],"failure": np.sum})
max_vs_failingdates.columns = max_vs_failingdates.columns.droplevel()
max_vs_failingdates["td"] = (max_vs_failingdates["max_date"] - max_vs_failingdates["failing_date"]) / np.timedelta64(1, 'D')
print
print "dt in days between first failure and end of measurement :"
print max_vs_failingdates["td"].value_counts()
print
print "n failures"
print max_vs_failingdates["sum"].value_counts()


In [ ]:
print "looking at weird failures"
weird_devices = max_vs_failingdates[max_vs_failingdates["td"] > 0]
weirdos = failing_devices_t.reset_index(level=1).ix[set(weird_devices.index)]
print weirdos.set_index("date",append=True).unstack(level="device").to_string()

 - indetified a list of devices, which are still measured after having failed.
 
three hypothesis:
 - The device is still functionnal after maintenance
 - The failure was a fluke
 - The measurement thereafter are false
 
 ==> if we cannot distinguish between these hypothesis, need to remove these devices from the dataset 